# Mobile Kinematic
<!-- Maintainer: Dang Tran: dangtm24@gmail.com -->

## Table of Contents

* [1. Wheel kinematic](#01)
* [2. Unicycle](#02)
  * [2.1 Wheel Kinematic Analysis](#021)
  * [2.2 Code](#022)
* [3. Differential-Drive Model](#03)
  * [3.1 Wheel Kinematic Analysis](#031)
  * [3.2 Code](#032)
* [4. Ackermann Steering model](#04)
  * [4.1 Wheel Kinematic Analysis](#041)
  * [4.2 Code](#042)
* [5. Reference](#05)

## Wheel kinematic <a class="anchor" id="01"></a>
While kinematics is a study of the mathematics of general solid body motion without considering the forces that affect the motion, wheel kinematic focus on mathematical equations involving with wheel designs. 

The most common questions when analyzing wheel-kinematic are:

- what is the configuration representation?
- what is the chasis configuration of the model?
- what is the velocity kinematic of the model?

Examples of Wheel kinematic analysis belows:

- Unicycle model
- Differential-Drive model
- Ackermann Steering model




## Unicycle <a class="anchor" id="02"></a>

<img src="img/unicycle_model.png" width="500"/>


Dealing with the displacement and velocities of the two wheels of a differential drive robot is messy. A preferred model is that of a unicycle, where we can think of the robot as having one wheel that can move with a desired velocity ($V$) at a specified heading ($\phi$). Having the equations to translate between the unicycle model and our wheel velocities is what allows us to simplify the robot with the unicycle model. We have seen how to take measured wheel displacements to calculate the new robot pose. Now we do the reverse and calculate the desired wheel velocities from the unicycle model.



### Wheel Kinematic Analysis <a class="anchor" id="021"></a>

The unicycle model contains a wheel with radius $r$. The chasis is the projection of wheel-center to the moving plane. For representing configuration of the chasis, we use 3DOF representation:

$$ q = 
    \begin{bmatrix} 
        x \\
        y \\
        \phi
    \end{bmatrix}
$$


where $x,y$ are initial coordinates of chasis on the plane, $\phi$ is initial angle of chasis w.r.t y-axis.


The velocity kinematic of the model can be derive

$$ V = \dot{q} = 
        \begin{bmatrix} 
                v \cdot cos(\phi) \\
                v \cdot sin(\phi) \\
                \omega         
        \end{bmatrix}
               =
        \begin{bmatrix} 
                v_x \\
                v_y \\
                \omega         
        \end{bmatrix}
$$

where $v$ is linear velocity, $\omega$ is angular velocity.

The velocity can be used to update new configuration by

$$ 
   X_{new} = X_{old} + V \cdot t
$$

where $t$ is time.


To integrate 2 additional parameters for controlling $u_1$ (forward-backward driving speed), $u_2$ (rotation turning speed), we change the equation above into

$$
\dot{q} = 
        \begin{bmatrix} 
                r \cdot cos(\phi) &     0 \\
                r \cdot sin(\phi) &     0 \\
                0                 &     1         
        \end{bmatrix}

        \begin{bmatrix} 
                u_1 \\
                u_2
        \end{bmatrix}
$$

where $r$ is radius of the wheel, $u_1$, $u_2$ are control parameters. 


### Code <a class="anchor" id="022"></a>

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt


def plot_arrow(q, length=0.01, width=0.005, fc="r", ec="k"):  # pragma: no cover
    """
    Plot arrow
    """
    x = q[0]
    y = q[1]
    yaw = q[2]

    if not isinstance(x, float):
        for (ix, iy, iyaw) in zip(x, y, yaw):
            plot_arrow(ix, iy, iyaw)
    else:
        plt.arrow(x, y, length * math.cos(yaw), length * math.sin(yaw),
                  fc=fc, ec=ec, head_width=width, head_length=width)
        plt.plot(x, y)


def main():

    # Initial position
    initial_x = .0
    initial_y = .0
    initial_angle_rad = math.pi / 6.
    q = np.array([
        initial_x,
        initial_y,
        initial_angle_rad
    ])
    radius = 0.5

    u1 = 0.1
    u2 = 0.05

    control_vector = np.array([
        u1,
        u2
    ])

    A = np.array([
        [radius * np.cos(q[2]), 0],
        [radius * np.sin(q[2]), 0],
        [0, 1]
    ])

    V = np.dot(A, control_vector)

    print(q)
    fig, ax = plt.subplots()
    plot_arrow(q)

    t = 10
    for i in range(t):
        q = q + V * 1
        plot_arrow(q)
    plt.show()


if __name__ == "__main__":
    main()


## Differential-Drive Model <a class="anchor" id="03"></a>

<img src="img/differiential_dirve_mode.png" width="500"/>

The differential-drive robot, or diff-drive, is perhaps the simplest wheeled
mobile robot architecture. A diff-drive robot consists of two independently
driven wheels of radius $r$ that rotate about the same axis, as well as one or more
caster wheels, ball casters, or low-friction sliders that keep the robot horizontal.

The distance between 2 wheels is $2d$. Left-wheel will have rotation $\theta_L$ and right-wheel will have rotation $\theta_R$


### Wheel Kinematic Analysis <a class="anchor" id="031"></a>

The chasis of robot is center between 2 wheels. For representing configuration of the chasis, we use 3DOF representation:

$$ q = 
    \begin{bmatrix} 
        x \\
        y \\
        \phi
    \end{bmatrix}
$$


where $x,y$ are initial coordinates of chasis on the plane, $\phi$ is initial angle of chasis w.r.t y-axis.

The velocity kinematic of the model can be derive

$$ V = \dot{q} = 
        \begin{bmatrix} 
                v \cdot cos(\phi) \\
                v \cdot sin(\phi) \\
                \omega         
        \end{bmatrix}
               =
        \begin{bmatrix} 
                v_x \\
                v_y \\
                \omega         
        \end{bmatrix}
$$

where $v$ is linear velocity, $\omega$ is angular velocity.


There are 2 additional parameters for controling $\theta_L$ and $\theta_R$, called $u_L$ and $u_R$ respectively. To integrate these parameters into velocity kinematic, we change the equation above into

$$
\dot{q} = 
        \begin{bmatrix} 
                \frac{r}{2} \cdot cos(\phi)     &     \frac{r}{2} \cdot cos(\phi) \\
                \frac{r}{2} \cdot sin(\phi)     &     \frac{r}{2} \cdot sin(\phi) \\
                -\frac{r}{2d}                   &     \frac{r}{2d}
        \end{bmatrix}

        \begin{bmatrix} 
                u_L \\
                u_R
        \end{bmatrix}
$$

where $r$ is radius of the wheel, $u_L$, $u_R$ are angular velocity control of left-wheel, right-wheel, $d$ is the distance from chasis to each wheel.

### Code <a class="anchor" id="032"></a>

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt


def plot_arrow(q, ax, label, length=0.01, width=0.005, fc="r", ec="k"):  # pragma: no cover
    """
    Plot arrow
    """
    x = q[0]
    y = q[1]
    yaw = q[2]

    if not isinstance(x, float):
        for (ix, iy, iyaw) in zip(x, y, yaw):
            plot_arrow(ix, iy, iyaw)
    else:
        ax.arrow(x, y, length * math.cos(yaw), length * math.sin(yaw),
                 fc=fc, ec=ec, head_width=width, head_length=width)
        ax.plot(x, y)

    ax.set_xlabel(label)


def main(uL, uR, ax, label):

    # Initial position
    initial_x = .0
    initial_y = .0
    initial_angle_rad = math.pi / 6.
    q = np.array([
        initial_x,
        initial_y,
        initial_angle_rad
    ])
    radius = 0.5
    d = 20

    control_vector = np.array([
        uL,
        uR
    ])

    A = np.array([
        [(radius / 2.0) * np.cos(q[2]), (radius / 2.0) * np.cos(q[2])],
        [(radius / 2.0) * np.sin(q[2]), (radius / 2.0) * np.sin(q[2])],
        [-radius / (2.0 * d), radius / (2.0 * d)]
    ])

    V = np.dot(A, control_vector)

    plot_arrow(q, ax, label)

    t = 10
    for i in range(t):
        q = q + V * 1
        plot_arrow(q, ax, label)


if __name__ == "__main__":

    fig, axs = plt.subplots(2, 2)

    # Move forward
    uL = +0.1
    uR = +0.1
    main(uL, uR, axs[0][0], "Move forward")

    # Move backward
    uL = -0.1
    uR = -0.1
    main(uL, uR, axs[0][1], "Move backward")

    # Move left
    uL = -1
    uR = +1
    main(uL, uR, axs[1][0], "Move left")

    # Move right
    uL = +1
    uR = -1
    main(uL, uR, axs[1][1], "Move right")

    plt.show()


## Ackermann Steering model <a class="anchor" id="04"></a>

<img src="img/ackermann_steering_model.png" width="500"/>


The most familiar wheeled vehicle is a car, with two steered front wheels and
two fixed-heading rear wheels. To prevent slipping of the front wheels, they are
steered using Ackermann steering. 

4-wheel model is projected into 2-wheel simulation, one in the front (called "front steering wheel") that is turnable, and one in the back (called "back rear wheel") that fixed to the body. The length between 2 imaginary wheel is $l$. 



### Wheel Kinematic Analysis <a class="anchor" id="041"></a>


The chasis of robot is the center of "back rear wheel". For representing configuration of the chasis, we use 3DOF representation:

$$ q = 
    \begin{bmatrix} 
        x \\
        y \\
        \phi
    \end{bmatrix}
$$

where $x,y$ are initial coordinates of chasis on the plane, $\phi$ is initial angle of chasis w.r.t y-axis.

The velocity kinematic of the model can be derive

$$ V = \dot{q} = 
        \begin{bmatrix} 
                v \cdot cos(\phi) \\
                v \cdot sin(\phi) \\
                \omega         
        \end{bmatrix}
               =
        \begin{bmatrix} 
                v_x \\
                v_y \\
                \omega         
        \end{bmatrix}
$$

where $v$ is linear velocity, $\omega$ is angular velocity.

There are 2 additional parameters for controling: $v$ and $w$ for linear velocity and rotation velocity respectively. $\psi$ angle is identified based on $w$. To integrate these parameters into velocity kinematic, we change the equation above into

$$
\dot{q} = 
        \begin{bmatrix} 
                cos(\phi)     &     0 \\
                sin(\phi)     &     0 \\
                \frac{\tan(\psi)}{l}                   &     1
        \end{bmatrix}

        \begin{bmatrix} 
                v \\
                w
        \end{bmatrix}
$$

where $l$ is the length between 2 wheels, $\psi$ is angle of the front-wheel, $v$ is linear velocity, $w$ is angular velocity.

$\psi$ can be computed as follows:

$$ \psi = \tan^{-1}({\frac{l\cdot w}{v}})  $$ 


### Code <a class="anchor" id="042"></a>

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt


def plot_arrow(q, ax, label, length=0.01, width=0.005, fc="r", ec="k"):  # pragma: no cover
    """
    Plot arrow
    """
    x = q[0]
    y = q[1]
    yaw = q[2]

    if not isinstance(x, float):
        for (ix, iy, iyaw) in zip(x, y, yaw):
            plot_arrow(ix, iy, iyaw)
    else:
        ax.arrow(x, y, length * math.cos(yaw), length * math.sin(yaw),
                 fc=fc, ec=ec, head_width=width, head_length=width)
        ax.plot(x, y)

    ax.set_xlabel(label)


def main(v, w, ax, label):

    # Initial position
    initial_x = .0
    initial_y = .0
    initial_angle_rad = math.pi / 6.
    q = np.array([
        initial_x,
        initial_y,
        initial_angle_rad
    ])
    radius = 0.5
    l = 50

    control_vector = np.array([
        v,
        w
    ])

    A = np.array([
        [np.cos(q[2]), 0],
        [np.sin(q[2]), 0],
        [np.tan(np.arctan((l * w) / v)) / l, 1]
    ])

    V = np.dot(A, control_vector)

    plot_arrow(q, ax, label)

    t = 10
    for i in range(t):
        q = q + V * 1
        plot_arrow(q, ax, label)


if __name__ == "__main__":

    fig, axs = plt.subplots(2, 2)

    # Move forward
    v = +0.1
    w = 0
    main(v, w, axs[0, 0], "Move forward")

    # Move backward
    v = -0.1
    w = 0
    main(v, w, axs[0, 1], "Move backward")

    # Move left
    v = +0.1
    w = 0.03
    main(v, w, axs[1, 0], "Move Left")

    # Move right
    v = +0.1
    w = -0.03
    main(v, w, axs[1, 1], "Move Right")
    plt.show()


## *Reference:* <a class="anchor" id="05"></a>